In [1]:
% matplotlib inline
% load_ext autoreload
% autoreload 2

In [2]:
from __future__ import print_function
import tensorflow as tf
import numpy as np
from deep_visual_odometry.kitti_utils import KITTIdata
from matplotlib import pyplot as plt

C:\Users\keert\Anaconda3\lib\site-packages\h5py\__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [3]:
basedir = 'dataset'

In [4]:
from deep_visual_odometry.odometry_model import OdomModel

In [5]:
batch_size = 256         # Sequences per batch
num_steps = 8         # Number of sequence steps per batch
rnn_size = 64          # Size of hidden layers in rnn_cell
num_layers = 2           # Number of hidden layers
learning_rate = 0.001

In [6]:
kitti_data = KITTIdata(basedir, 
                       sequences = ['01'],
                       sequence_len = num_steps,
                       val_frac = 0,
                       test_frac = 0,
                       img_size=(150,50))

completed load sequence 01 data


In [7]:
mean_velo = np.mean(kitti_data.velocities['01'],axis = 0)
std_velo = np.std(kitti_data.velocities['01'],axis = 0)

mean_poses = np.mean(kitti_data.poses['01'],axis = 0)
std_poses = np.std(kitti_data.poses['01'],axis = 0)

In [8]:
kitti_data.velocities['01'] = kitti_data.velocities['01']-mean_velo
kitti_data.velocities['01'] /= std_velo

In [9]:
kitti_data.poses['01'] = kitti_data.poses['01']-mean_poses
kitti_data.poses['01'] /= std_poses

In [10]:
model = OdomModel(batch_size = 256, 
                  num_steps = 8,
                  cell_type = 'GRU',
                  rnn_size = 128,
                  num_layers = 2,
                  learning_rate = 0.001)
model.train(kitti_data, max_count = 10000, save_every_n = 1000)

output size (256, 8, 128)
seq_output shape (256, 8, 128)
x (2048, 128)
mse_w (128, 3)
shape of logits (256, 8, 3)
shape of targets (256, 8, 3)
step: 500  loss: 0.4441  0.1077 sec/batch
step: 1000  loss: 0.3274  0.0987 sec/batch
step: 1500  loss: 0.3460  0.1067 sec/batch
step: 2000  loss: 0.2915  0.1018 sec/batch
step: 2500  loss: 0.2892  0.1017 sec/batch
step: 3000  loss: 0.2743  0.0987 sec/batch
step: 3500  loss: 0.2560  0.1207 sec/batch
step: 4000  loss: 0.2645  0.0967 sec/batch
step: 4500  loss: 0.2799  0.0997 sec/batch
step: 5000  loss: 0.2642  0.1077 sec/batch
step: 5500  loss: 0.2376  0.0987 sec/batch
step: 6000  loss: 0.2492  0.1067 sec/batch
step: 6500  loss: 0.2594  0.1028 sec/batch
step: 7000  loss: 0.2318  0.0917 sec/batch
step: 7500  loss: 0.2344  0.1087 sec/batch
step: 8000  loss: 0.1997  0.1027 sec/batch
step: 8500  loss: 0.2357  0.1217 sec/batch
step: 9000  loss: 0.2194  0.1047 sec/batch
step: 9500  loss: 0.2252  0.0997 sec/batch
step: 10000  loss: 0.2103  0.1087 sec/bat

In [ ]:
kitti_data.dataset_len['01']

In [ ]:
inputs_train, velocities_train, poses_train = kitti_data.load_data_train()

In [ ]:
inputs_train.shape